In [47]:
from apps.source.models import Category, SourceCategory
import requests
import bs4

from etl.category import ExtractionByCategory

In [85]:
category = Category.objects.first()

In [86]:
category.name

'Politica'

In [87]:
articles = ExtractionByCategory().extract(category)

In [88]:
for article in articles:
    print(article.title , '|' ,article.source)

 Elecciones 2021: ¿Es posible que votaciones se prolonguen por 2 días por la pandemia?  Expertos evalúan las ventajas y desventajas, así como alternativas y otras previsiones para las elecciones en un contexto de pandemiaSuscriptor Digital | El Comercio
 Keiko Fujimori: Todas las veces que se pronunció desde que fue excarcelada Tras la última carta que la excandidata presidencial envió a los congresistas de Fuerza Popular, el fiscal José Domingo Pérez ha solicitado al Poder Judicial advertirle sobre un supuesto incumplimiento de reglas de conducta. | El Comercio
 Edgar Alarcón, el futuro del congresista con dos denuncias constitucionales La Subcomisión de Acusaciones Constitucionales es la instancia que deberá revisar el caso del congresista Alarcón, denunciado por enriquecimiento ilícito y peculado doloso. ¿Cuáles son los plazos que rigen para este caso?Suscriptor Digital | El Comercio
 Martín Vizcarra: Los escenarios legales por los contratos de su cuñado con el Estado Según especial

In [51]:
sources_link = SourceCategory.objects.filter(category=category)

In [52]:
sources_link

<QuerySet [<SourceCategory: SourceCategory object (3)>, <SourceCategory: SourceCategory object (12)>]>

In [53]:
source_category = sources_link[1]
source_category

<SourceCategory: SourceCategory object (12)>

In [54]:
source = source_category.source
source

<Source: Peru 21>

In [55]:
url = source_category.url
url

'https://peru21.pe/politica/'

In [56]:
response = requests.get(url)

In [57]:
response.text

'<!DOCTYPE html><html itemscope="" itemType="http://schema.org/WebPage" lang="es"><head><meta charSet="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="lang" content="es"/><meta name="resource-type" content="document"/><meta content="global" name="distribution"/><meta name="robots" content="index, follow"/><meta name="GOOGLEBOT" content="index follow"/><meta name="Author" content="Peru21"/><title>Noticias de Política | Política | NOTICIAS PERU21 PERÚ</title><link rel="preconnect dns-prefetch" href="//cdnc.peru21.pe"/><link rel="preconnect dns-prefetch" href="//cdna.peru21.pe"/><link rel="preconnect dns-prefetch" href="//d1r08wok4169a5.cloudfront.net"/><link rel="preconnect dns-prefetch" href="//elcomercio-elcomercio-prod.cdn.arcpublishing.com"/><link rel="preconnect dns-prefetch" href="//arc-anglerfish-arc2-prod-elcomercio.s3.amazonaws.com"/><link rel="preconnect dns-prefetch" href="//s.go-mpu

In [68]:
source.category_articles_link = 'featured-story__title'

In [69]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')
articles_links = soup.select('.'+source.category_articles_link)
articles_links

[<h2 class="featured-story__title overflow-hidden mb-10 line-h-xs" itemprop="name"><a class="featured-story__title-link title-xs line-h-sm" href="/politica/reabrir-el-fronton-el-ministro-rodriguez-despierta-un-debate-innecesario-noticia/" itemprop="url">Reabrir El Frontón: “El ministro Rodríguez despierta un debate innecesario”</a></h2>,
 <h2 class="featured-story__title overflow-hidden mb-10 line-h-xs" itemprop="name"><a class="featured-story__title-link title-xs line-h-sm" href="/politica/alejandro-toledo-odebrecht-alejandro-toledo-arguye-que-no-puede-ser-extraditado-porque-no-ha-sido-imputado-de-ningun-delito-en-peru-noticia/" itemprop="url">Alejandro Toledo arguye que no puede ser extraditado porque no ha sido acusado de ningún delito en el Perú</a></h2>,
 <h2 class="featured-story__title overflow-hidden mb-10 line-h-xs" itemprop="name"><a class="featured-story__title-link title-xs line-h-sm" href="/politica/congreso-ministro-de-defensa-pide-mesura-al-congreso-por-interpelacion-a-t

In [70]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')
articles_links = soup.select('.'+source.category_articles_link)
articles = [article.find('a')['href'] for article in articles_links]


top_articles = []

for i in range(min(10, len(articles))):
    top_articles.append(articles[i])
top_articles

['/politica/reabrir-el-fronton-el-ministro-rodriguez-despierta-un-debate-innecesario-noticia/',
 '/politica/alejandro-toledo-odebrecht-alejandro-toledo-arguye-que-no-puede-ser-extraditado-porque-no-ha-sido-imputado-de-ningun-delito-en-peru-noticia/',
 '/politica/congreso-ministro-de-defensa-pide-mesura-al-congreso-por-interpelacion-a-tres-ministros-nndc-noticia/',
 '/politica/onpe-senala-imposibilidad-de-continuar-con-financiamiento-a-partidos-del-congreso-disuelto-nndc-noticia/',
 '/politica/reforma-politica-manuel-merino-de-lama-hemos-pedido-a-comision-de-constitucion-mejorar-dictamen-sobre-inmunidad-nndc-noticia/',
 '/politica/edgar-alarcon-frente-amplio-pide-su-renuncia-a-la-presidencia-de-la-comision-de-fiscalizacion-noticia/',
 '/politica/feriados-2020-poder-ejecutivo-modifica-decreto-sobre-dias-no-laborables-y-el-27-de-julio-no-sera-feriado-nndc-noticia/',
 '/politica/hector-becerril-fiscal-de-la-nacion-presento-dos-denuncias-constitucionales-contra-exlegislador-fujimorista-fuer

In [71]:
import re 
is_well_formed_url = re.compile(r'^https?://.+/.+$')
is_root_path = re.compile(r'^/.+$')

def build_link(host, link):
    if is_well_formed_url.match(link):
        return link
    elif is_root_path.match(link):
        return '{host}{uri}'.format(host=host, uri=link[1:len(link)])
    else:
        link

In [72]:
for i in range(len(top_articles)):
    top_articles[i] = build_link(source.host, top_articles[i])

In [73]:
top_articles

['https://peru21.pe/politica/reabrir-el-fronton-el-ministro-rodriguez-despierta-un-debate-innecesario-noticia/',
 'https://peru21.pe/politica/alejandro-toledo-odebrecht-alejandro-toledo-arguye-que-no-puede-ser-extraditado-porque-no-ha-sido-imputado-de-ningun-delito-en-peru-noticia/',
 'https://peru21.pe/politica/congreso-ministro-de-defensa-pide-mesura-al-congreso-por-interpelacion-a-tres-ministros-nndc-noticia/',
 'https://peru21.pe/politica/onpe-senala-imposibilidad-de-continuar-con-financiamiento-a-partidos-del-congreso-disuelto-nndc-noticia/',
 'https://peru21.pe/politica/reforma-politica-manuel-merino-de-lama-hemos-pedido-a-comision-de-constitucion-mejorar-dictamen-sobre-inmunidad-nndc-noticia/',
 'https://peru21.pe/politica/edgar-alarcon-frente-amplio-pide-su-renuncia-a-la-presidencia-de-la-comision-de-fiscalizacion-noticia/',
 'https://peru21.pe/politica/feriados-2020-poder-ejecutivo-modifica-decreto-sobre-dias-no-laborables-y-el-27-de-julio-no-sera-feriado-nndc-noticia/',
 'htt

In [74]:
article_link = top_articles[0]

In [75]:
article_link

'https://peru21.pe/politica/reabrir-el-fronton-el-ministro-rodriguez-despierta-un-debate-innecesario-noticia/'

In [76]:
response2 = requests.get(article_link)

In [77]:
response2.text

'<!DOCTYPE html><html itemscope="" itemType="http://schema.org/WebPage" lang="es"><head><meta charSet="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="lang" content="es"/><meta name="resource-type" content="document"/><meta content="global" name="distribution"/><meta name="robots" content="index, follow"/><meta name="GOOGLEBOT" content="index follow"/><meta name="Author" content="Peru21"/><meta name="DC.title" lang="es" content="Política: Reabrir El Frontón: “El ministro Rodríguez despierta un debate inneces | NOTICIAS PERU21 PERÚ"/><meta name="DC.description" lang="es" content="Especialista en cárceles y seguridad opinan sobre propuesta del Ministerio del Interior "/><meta name="DC.subject" lang="es" content="Noticias, Peru21, Peru, Mundo, Deportes, Internacional, Tecnologia, Diario, Cultura, Ciencias, Economía, Opinión"/><meta name="DC.creator" content="NOTICIAS PERU21"/><meta name="DC.publ

In [78]:
from apps.article.models import Article

In [79]:
soup2 = bs4.BeautifulSoup(response2.text, 'html.parser')

In [80]:
soup2.select('.'+source.article_title_class)[0].text

' Reabrir El Frontón: “El ministro Rodríguez despierta un debate innecesario”'

In [81]:
soup2.select('.'+source.article_body_class)

[<div class="story-content__content position-relative flex flex-row-reverse false" id="contenedor"><div class="story-content__icon-list no-mobile non-tablet md:pt-20 md:pb-0 md:pr-20"><ul class="story-content__list"><li class="story-content__item mb-20 position-relative"><a class="story-content__link text-gray-200" data-social="share-print" href="" itemprop="url" role="button" title="Imprimir noticia"><i aria-hidden="true" class="icon-print story-content__icon title-xl"></i></a></li><li class="story-content__item mb-20 position-relative"><a class="story-content__link text-gray-200" data-social="share-more" href="" itemprop="url" role="button" title="Enlaces para compartir"><i aria-hidden="true" class="icon-link story-content__icon title-xl"></i></a><ul class="story-content__list-more bg-white position-absolute flex hidden top-0 justify-between p-10"><li class="story-content__item flex justify-center"><a class="story-content__more-link flex items-center" data-social="share-social" href=

In [82]:
soup2.select('.'+source.article_image_class)[0]['src']

'https://peru21.pe/resizer/-9tnao9XoFf9JwAMIAAbVPtTYkw=/580x330/smart/cloudfront-us-east-1.images.arcpublishing.com/elcomercio/XBMSEV5CNREOJEPILWLRPHUWPU.jpg'

In [83]:
source.article_image_class

'story-content__visual--image'

In [84]:
source.article_image_class = 'story-content__visual--image'